```{autolink-concat}
```

::::{margin}
:::{card} Indexed free symbols
TR-008
^^^
This report has been implemented in [ampform#111](https://github.com/ComPWA/ampform/issues/111). Additionally, [tensorwaves#427](https://github.com/ComPWA/tensorwaves/issues/427) makes it possible to lambdify {class}`sympy.Expr <sympy.core.expr.Expr>` with {class}`~sympy.tensor.indexed.Indexed` symbols directly.
+++
✅&nbsp;[ampform#111](https://github.com/ComPWA/ampform/issues/111)
:::
::::

# `Indexed` free symbols

In [ ]:
%pip install -q sympy==1.8

In [TR-005](005.ipynb), we made use of {doc}`indexed symbols <sympy:modules/tensor/indexed>` to create a $\boldsymbol{K}$-matrix. The problem with that approach is that {class}`~sympy.tensor.indexed.IndexedBase` and their resulting {class}`~sympy.tensor.indexed.Indexed` instances when taking indices behave strangely in an expression tree.

The following {class}`~sympy.core.expr.Expr` uses a {class}`~sympy.core.symbol.Symbol` and a elements in {class}`~sympy.tensor.indexed.IndexedBase`s (an {class}`~sympy.tensor.indexed.Indexed` instance):

In [ ]:
import sympy as sp

x = sp.Symbol("x")
c = sp.IndexedBase("c")
alpha = sp.IndexedBase("alpha")
expression = c[0, 1] + alpha[2] * x
expression

x*alpha[2] + c[0, 1]

Although seemingly there are just **three** {attr}`~sympy.core.basic.Basic.free_symbols`, there are actually **five**:

In [ ]:
expression.free_symbols

{alpha, alpha[2], c, c[0, 1], x}

This becomes problematic when using {func}`~sympy.utilities.lambdify.lambdify`, particularly through {func}`symplot.prepare_sliders`.

In addition, while `c[0, 1]` and `alpha[2]` are {class}`~sympy.tensor.indexed.Indexed` as expected, `alpha` and `c` are {class}`~sympy.core.symbol.Symbol`s, not {class}`~sympy.tensor.indexed.IndexedBase`:

In [ ]:
{s: type(s) for s in expression.free_symbols}

{c: sympy.core.symbol.Symbol,
 alpha[2]: sympy.tensor.indexed.Indexed,
 x: sympy.core.symbol.Symbol,
 c[0, 1]: sympy.tensor.indexed.Indexed,
 alpha: sympy.core.symbol.Symbol}

The {doc}`expression tree <sympy:tutorials/intro-tutorial/manipulation>` partially explains this behavior:

In [ ]:
import graphviz

dot = sp.dotprint(expression)
graphviz.Source(dot);

![](https://user-images.githubusercontent.com/29308176/164993648-13c6b74a-b85f-4492-aaf2-c64cdc30e345.svg)

We would like to collapse the nodes under `c[0, 1]` and `alpha[2]` to two single {class}`~sympy.core.symbol.Symbol` nodes that are **still nicely rendered as $c_{0,1}$ and $\alpha_2$**. The following function does that and converts the `[]` into subscripts. It does that in such a way that the name of the {class}`~sympy.core.symbol.Symbol` remains as short as possible, that is, short enough that it still renders nicely as LaTeX:

In [ ]:
from sympy.printing.latex import translate


def to_symbol(idx: sp.Indexed) -> sp.Symbol:
    base_name, _, _ = str(idx).rpartition("[")
    subscript = ",".join(map(str, idx.indices))
    if len(idx.indices) > 1:
        base_name = translate(base_name)
        subscript = "_{" + subscript + "}"
    return sp.Symbol(f"{base_name}{subscript}")

Next, we use {meth}`~sympy.core.basic.Basic.subs` to substitute the nodes `c[0, 1]` and `alpha[2]` with these {class}`~sympy.core.symbol.Symbol`s:

In [ ]:
def replace_indexed_symbols(expression: sp.Expr) -> sp.Expr:
    return expression.subs({
        s: to_symbol(s) for s in expression.free_symbols if isinstance(s, sp.Indexed)
    })

And indeed, the expression tree has been simplified correctly!

In [ ]:
new_expression = replace_indexed_symbols(expression)
dot = sp.dotprint(new_expression)
graphviz.Source(dot);

![](https://user-images.githubusercontent.com/29308176/164993649-47231cf6-0ee2-4eed-a122-633e2cf5db1a.svg)